In [1]:
import os
import requests
from bs4 import BeautifulSoup
import json

# Function to create a directory for the city
def create_city_directory(city):
    if not os.path.exists(city):
        os.makedirs(city)

# Function to extract doctor details from a given city page
def extract_doctor_details(url, city):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the doctor listing elements
    listings = soup.find_all(id=lambda x: x and x.startswith('doctor-listing'))

    doctors_info = []
    
    for listing in listings:
        doctors = listing.find_all('div', class_='row shadow-card')
        for doctor in doctors:
            name_tag = doctor.find('h3')
            if name_tag:
                name = name_tag.text.strip()
                profile_url = doctor.find('a', class_='dr_profile_opened_from_listing')['href']
                specialty = doctor.find('p', class_='mb-0 mt-10 text-sm').text.strip()
                experience = doctor.find_all('p', class_='text-bold text-sm')[0].text.strip() if doctor.find_all('p', class_='text-bold text-sm') else 'N/A'
                
                doctor_info = {
                    'name': name,
                    'profile_url': profile_url,
                    'specialty': specialty,
                    'experience': experience
                }
                doctors_info.append(doctor_info)

    return doctors_info

# Function to save doctor details to a JSON file
def save_doctors_to_file(city, doctors_info):
    with open(f"{city}/{city}_doctors.json", 'w') as json_file:
        json.dump(doctors_info, json_file, indent=4)

def main():
    # Read URLs from the text file
    with open('pediatric-neurosurgeon.txt', 'r') as file:
        links = file.readlines()

    for link in links:
        link = link.strip()
        city = link.split('/')[-2]  # Assuming the city is part of the URL
        create_city_directory(city)
        doctors_info = extract_doctor_details(link, city)
        save_doctors_to_file(city, doctors_info)

if __name__ == '__main__':
    main()
